In [1]:
# install libraries
import pandas as pd 
import os 
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

In [2]:
# Function to check shape and unique values of a dataframe
def inital_shape_unique(df):
    
    # Check dataframe shape
    print(f"Accident dataframe has a shape of {df.shape}")
    
    print(df.dtypes)

    # Check for unique values in each columns
    for i in df.columns:
        print("-----------------------------------")
        print(f"{i} has {df[i].nunique()} unique values")
        print(f"{df[i].unique()}")
        print(f"Null Presents: {df[i].isnull().any()}")

In [3]:
# Create a df for all the tables 

accident_path = "./Resources/ACCIDENT.csv"
accident_event_path = "./Resources/ACCIDENT_EVENT.csv"
accident_location_path = "./Resources/ACCIDENT_LOCATION.csv"
atmospheric_cond_path = "./Resources/ATMOSPHERIC_COND.csv"
node_path = "./Resources/NODE.csv"
person_path = "./Resources/PERSON.csv"
road_surface_cond_path = "./Resources/ROAD_SURFACE_COND.csv"
sub_dca_path = "./Resources/SUB_DCA.csv"
vehicle_path = "./Resources/VEHICLE.csv"

accident_df = pd.read_csv(accident_path)
accident_event_df = pd.read_csv(accident_event_path)
accident_location_df = pd.read_csv(accident_location_path)
atmospheric_df = pd.read_csv(atmospheric_cond_path)
node_df = pd.read_csv(node_path)
person_df = pd.read_csv(person_path)
road_surface_df = pd.read_csv(road_surface_cond_path)
sub_dca_df = pd.read_csv(sub_dca_path)
vehicle_df = pd.read_csv(vehicle_path)

## Inital questions about the topic.

- Time period of the data goes from .... to ....

- Check the number of accidents within time period

- What time does most accidents happen?

- What is the most occuring accident type?

- Which day has the most accidents happen?

- The average number of police attendance related to number killed/injuried?

- What accident type kills/injures more people?

- What speed zone has more accidents? what type of accidents for each speed zone? 

- Does speed zone correlate with severity of accident?

- Does road geometry correlate with amount of crashes?



In [4]:
accident_df.head()

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,ACCIDENT_TYPE,ACCIDENT_TYPE_DESC,DAY_OF_WEEK,DAY_WEEK_DESC,DCA_CODE,DCA_DESC,LIGHT_CONDITION,...,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_NOT_INJ,NO_PERSONS,POLICE_ATTEND,ROAD_GEOMETRY,ROAD_GEOMETRY_DESC,SEVERITY,SPEED_ZONE,RMA
0,T20120000060,2012-01-01,19:40:00,6,Vehicle overturned (no collision),1,Sunday,184,OUT OF CONTROL ON CARRIAGEWAY (ON BEND),1,...,1,0,0,1,1,2,T intersection,2,60,Local Road
1,T20120000028,2012-01-01,04:00:00,4,Collision with a fixed object,1,Sunday,183,OFF LEFT BEND INTO OBJECT/PARKED VEHICLE,5,...,1,0,0,1,1,2,T intersection,2,100,Local Road
2,T20120000021,2012-01-01,07:30:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,1,...,0,2,1,3,1,5,Not at intersection,3,50,Local Road
3,T20120000050,2012-01-01,15:00:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,1,...,0,1,0,1,1,5,Not at intersection,3,999,Local Road
4,T20120000056,2012-01-01,16:15:00,4,Collision with a fixed object,1,Sunday,183,OFF LEFT BEND INTO OBJECT/PARKED VEHICLE,1,...,2,0,0,2,1,2,T intersection,2,100,Local Road


In [5]:
accident_event_df.head()

,ACCIDENT_NO,EVENT_SEQ_NO,EVENT_TYPE,EVENT_TYPE_DESC,VEHICLE_1_ID,VEHICLE_1_COLL_PT,VEHICLE 1 COLL PT DESC,VEHICLE_2_ID,VEHICLE_2_COLL_PT,VEHICLE 2 COLL PT DESC,PERSON_ID,OBJECT_TYPE,OBJECT_TYPE_DESC
0,T20120003962,1,C,Collision,A,1,Right front corner,B,5,Left front corner,NaN,99,Not Applicable
1,T20120003991,1,3,Ran off carriageway,A,9,Not known or Not Applicable,NaN,NaN,NaN,NaN,99,Not Applicable
2,T20120003991,2,C,Collision,A,1,Right front corner,NaN,NaN,NaN,NaN,6,Traffic sign (no parking/no standing)
3,T20120003991,3,C,Collision,A,9,Not known or Not Applicable,NaN,NaN,NaN,NaN,2,Tree (shrub/scrub)
4,T20120003998,1,C,Collision,A,2,Right side (forwards),B,5,Left front corner,NaN,99,Not Applicable


###### Inital questions about accident event dataframe

- How many accidents are there?
- What is the biggest sequence of accidents? 
- Most occuring event type?
- Most occuring object collision?
- Percentage of crashes that involved a second indivudal/car?

In [6]:
accident_location_df.head()

,ACCIDENT_NO,NODE_ID,ROAD_ROUTE_1,ROAD_NAME,ROAD_TYPE,ROAD_NAME_INT,ROAD_TYPE_INT,DISTANCE_LOCATION,DIRECTION_LOCATION
0,T20120000060,248602,9999.0,LINMAX,COURT,WALLACE,AVENUE,0.0,NW
1,T20120000082,41011,5784.0,DORSET,ROAD,ALLAMBANAN,DRIVE,0.0,S
2,T20120000090,273187,9999.0,CHURCHILL PARK,DRIVE,PARKSIDE,BOULEVARD,197.0,E
3,T20120000093,20746,2510.0,PRINCES,HIGHWAY,IRVINES,ROAD,0.0,SW
4,T20120000104,248609,9999.0,DAVIS,ROAD,LEMARNE,ROAD,0.0,W


###### Inital questions about accident location dataframe

- Most occurring road type accidents
- Does direction location have an influence on the amount of crashes? Driving W or E (towards the sun have more accidents?) 

In [7]:
atmospheric_df.head()

,ACCIDENT_NO,ATMOSPH_COND,ATMOSPH_COND_SEQ,ATMOSPH_COND_DESC
0,T20130006765,1,1,Clear
1,T20130006774,1,1,Clear
2,T20130006803,1,1,Clear
3,T20130006824,9,1,Not known
4,T20130006833,1,1,Clear


###### Inital questions for Atmospheric dataframe 

- What condidtion do accident occur the most in?

In [8]:
node_df.head()

,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH
0,T20220021115,725433,N,2.638082e+06,2.366974e+06,LATROBE,RURAL_VICTORIA,-38.188345,146.576115,NaN
1,T20220016471,726743,N,2.504398e+06,2.403846e+06,STONNINGTON,MELB_URBAN,-37.866454,145.049989,NaN
2,T20220011545,726774,O,2.652318e+06,2.389261e+06,WELLINGTON,RURAL_VICTORIA,-37.985367,146.733949,NaN
3,T20220009098,726776,N,2.577329e+06,2.522066e+06,BENALLA,RURAL_VICTORIA,-36.797963,145.866604,NaN
4,T20220017520,726779,N,2.414327e+06,2.386234e+06,GOLDEN PLAINS,SMALL_TOWNS,-38.021179,144.024292,NaN


###### Inital question for Node dataframe
- What urban name has the most accidents?
- Can we map the latitude and longitude with severity, etc? 
- What does Node stand for? Drop it due to no context for it? Drop AMG colums aswell?
- Can we fill in postcode through the use of latitude and longitude?

In [9]:
person_df.head()

,ACCIDENT_NO,PERSON_ID,VEHICLE_ID,SEX,AGE_GROUP,INJ_LEVEL,INJ_LEVEL_DESC,SEATING_POSITION,HELMET_BELT_WORN,ROAD_USER_TYPE,ROAD_USER_TYPE_DESC,LICENCE_STATE,TAKEN_HOSPITAL,EJECTED_CODE
0,T20220027723,B,B,M,50-59,3,Other injury,D,9,2,Drivers,V,NaN,0.0
1,T20210018500,A,A,M,30-39,3,Other injury,D,1,2,Drivers,V,NaN,0.0
2,T20210026317,01,B,M,70+,4,Not injured,OR,9,3,Passengers,NaN,NaN,9.0
3,T20220000143,A,A,F,65-69,3,Other injury,D,9,2,Drivers,V,NaN,0.0
4,T20170015017,A,A,M,40-49,4,Not injured,D,1,2,Drivers,V,NaN,0.0


###### Inital question for Person dataframe

- Percentage of accidents where indivudal had to be taken to hospital?
- Percentage of sex on accident occurence?
- Percentage of age group on accident occurence?
- Injury type change with sex/age group/seating position?

<b>ACCIDENT_NO</b>, <b>PERSON_ID</b>, VEHICLE_ID, SEX, AGE_GROUP, INJ_LEVEL, INJ_LEVEL_DESC, SEATING_POSITION, HELMET_BELT_WORN, ROAD_USER_TYPE_DESC, LICENSE_STATE, TAKEN_HOSPITAL, EJECTED_CODE

In [10]:
road_surface_df.head()

,ACCIDENT_NO,SURFACE_COND,SURFACE_COND_DESC,SURFACE_COND_SEQ
0,T20130012339,9,Unk.,1
1,T20130012360,1,Dry,1
2,T20130012370,1,Dry,1
3,T20130012375,1,Dry,1
4,T20130012390,2,Wet,1


###### Inital question about road surface dataframe

- Percentage of condition on accident occurance?
- Do wet condition lead to a longer sequence of accidents?

In [11]:
sub_dca_df.head()

,ACCIDENT_NO,SUB_DCA_CODE,SUB_DCA_SEQ,SUB_DCA_CODE_DESC
0,T20120023069,NRQ,1,Not Required
1,T20120023107,L04,1,Road curved at mid-block
2,T20120023147,N01,1,Intersection
3,T20120023151,NRQ,1,Not Required
4,T20120023171,NRQ,1,Not Required


###### Inital question for Sub DCA dataframe

- Do we need this?
- 

In [12]:
vehicle_df

,ACCIDENT_NO,VEHICLE_ID,VEHICLE_YEAR_MANUF,VEHICLE_DCA_CODE,INITIAL_DIRECTION,ROAD_SURFACE_TYPE,ROAD_SURFACE_TYPE_DESC,REG_STATE,VEHICLE_BODY_STYLE,VEHICLE_MAKE,...,TRAILER_TYPE,VEHICLE_COLOUR_1,VEHICLE_COLOUR_2,CAUGHT_FIRE,INITIAL_IMPACT,LAMPS,LEVEL_OF_DAMAGE,TOWED_AWAY_FLAG,TRAFFIC_CONTROL,TRAFFIC_CONTROL_DESC
0,T20120000009,A,1996.0,1.0,E,1.0,Paved,V,SEDAN,HOLDEN,...,H,WHI,ZZ,2.0,1,1.0,5,1.0,0.0,No control
1,T20120000012,A,2002.0,1.0,W,1.0,Paved,V,SEDAN,HOLDEN,...,H,BLU,ZZ,2.0,1,1.0,2,2.0,1.0,Stop-go lights
2,T20120000012,B,1988.0,2.0,S,1.0,Paved,V,SEDAN,TOYOTA,...,H,WHI,ZZ,2.0,5,9.0,4,1.0,1.0,Stop-go lights
3,T20120000013,A,1997.0,1.0,N,1.0,Paved,V,SEDAN,MITSUB,...,H,GLD,ZZ,2.0,5,1.0,4,1.0,0.0,No control
4,T20120000013,B,2010.0,2.0,N,1.0,Paved,V,WAGON,TOYOTA,...,H,BLK,ZZ,2.0,4,2.0,4,2.0,0.0,No control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304724,T20240006909,B,2018.0,1.0,W,1.0,Paved,V,SEDAN,SUZUKI,...,H,WHI,ZZ,2.0,5,2.0,1,2.0,0.0,No control
304725,T20240006923,A,2012.0,1.0,S,9.0,Not known,V,WAGON,JEEP,...,H,BLK,ZZ,2.0,F,2.0,5,2.0,99.0,Unknown
304726,T20240006923,B,2010.0,2.0,N,9.0,Not known,V,SEDAN,FORD,...,H,SIL,ZZ,2.0,F,2.0,5,2.0,99.0,Unknown
304727,T20240006955,A,2005.0,2.0,W,1.0,Paved,V,SEDAN,TOYOTA,...,H,BLU,ZZ,2.0,F,2.0,6,2.0,5.0,Ped. crossing


###### Inital question about Vehicle dataframe

- What brand has the most accidents?
- Does color affect accident occurance?
- Does Year manufacturing affect the accident occurance?
- Is there a noticeable group that has unverified rego?
- Does Veh body style affect accident occurance
- What brand has higher level of damage?
- Does direction affect accident occurance?

# accident_df 

In [13]:
# accident_df firsst look
inital_shape_unique(accident_df)

Accident dataframe has a shape of (167153, 23)
ACCIDENT_NO           object
ACCIDENT_DATE         object
ACCIDENT_TIME         object
ACCIDENT_TYPE          int64
ACCIDENT_TYPE_DESC    object
DAY_OF_WEEK            int64
DAY_WEEK_DESC         object
DCA_CODE               int64
DCA_DESC              object
LIGHT_CONDITION        int64
NODE_ID                int64
NO_OF_VEHICLES         int64
NO_PERSONS_KILLED      int64
NO_PERSONS_INJ_2       int64
NO_PERSONS_INJ_3       int64
NO_PERSONS_NOT_INJ     int64
NO_PERSONS             int64
POLICE_ATTEND          int64
ROAD_GEOMETRY          int64
ROAD_GEOMETRY_DESC    object
SEVERITY               int64
SPEED_ZONE             int64
RMA                   object
dtype: object
-----------------------------------
ACCIDENT_NO has 167153 unique values
['T20120000060' 'T20120000028' 'T20120000021' ... 'T20230022887'
 'T20230022886' 'T20230022860']
Null Presents: False
-----------------------------------
ACCIDENT_DATE has 4291 unique values
['2012-0

### Investigation on accident_df

In [14]:
# Investigate the difference between count of days vs day description 
for i in accident_df["DAY_OF_WEEK"].unique():
    print("---------------------------------------")
    print(i)
    unique_days = accident_df[["DAY_WEEK_DESC","DAY_OF_WEEK"]].loc[(accident_df["DAY_OF_WEEK"]==i)]["DAY_WEEK_DESC"].unique()
    count_days =accident_df[["DAY_WEEK_DESC","DAY_OF_WEEK"]].loc[(accident_df["DAY_OF_WEEK"]==i)].groupby("DAY_WEEK_DESC").sum()

    print(unique_days)
    print(count_days)                                                                          
    

---------------------------------------
1
['Sunday' 'Monday']
               DAY_OF_WEEK
DAY_WEEK_DESC             
Monday                5123
Sunday               15922
---------------------------------------
2
['Monday' 'Tuesday']
               DAY_OF_WEEK
DAY_WEEK_DESC             
Monday               35222
Tuesday              11150
---------------------------------------
3
['Tuesday' 'Wednesday']
               DAY_OF_WEEK
DAY_WEEK_DESC             
Tuesday              55437
Wednesday            17298
---------------------------------------
4
['Wednesday' 'Thursday']
               DAY_OF_WEEK
DAY_WEEK_DESC             
Thursday             22800
Wednesday            76832
---------------------------------------
5
['Thursday' 'Friday']
               DAY_OF_WEEK
DAY_WEEK_DESC             
Friday               30865
Thursday             98260
---------------------------------------
6
['Friday' 'Saturday']
               DAY_OF_WEEK
DAY_WEEK_DESC             
Friday              

In [15]:
# Check to see if DCA code lines up with victoria documented codes and checking descriptions aswell
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(accident_df.groupby(["DCA_CODE","DCA_DESC"]).count())


                                                             ACCIDENT_NO  \
DCA_CODE DCA_DESC                                                          
100      PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIGHT.          5315   
101      PED EMERGES FROM IN FRONT OF PARKED OR STATIONA...          830   
102      FAR SIDE. PED HIT BY VEHICLE FROM THE LEFT                 3461   
103      PED PLAYING/LYING/WORKING/STANDING ON CARRIAGEWAY.          764   
104      PED WALKING WITH TRAFFIC                                    362   
105      PED WALKING AGAINST TRAFFIC.                                177   
106      VEH STRIKES PED ON FOOTPATH/MEDIAN/TRAFFIC ISLAND.          420   
107      PED ON FOOTHPATH STRUCK BY VEHENTERING/LEAVING ...          772   
108      PED STRUCK WALKING TO/FROM OR BOARDING/ALIGHTIN...          421   
109      ANY MANOEUVRE INVOLVING PED NOT INCLUDED IN DCA...         1232   
110      CROSS TRAFFIC(INTERSECTIONS ONLY)                         11929   
111      RIG

In [16]:
#checking speed zone value of 777, 888 & 999

accident_df.loc[(accident_df["SPEED_ZONE"]==888)|(accident_df["SPEED_ZONE"]==999)|(accident_df["SPEED_ZONE"]==777)]

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,ACCIDENT_TYPE,ACCIDENT_TYPE_DESC,DAY_OF_WEEK,DAY_WEEK_DESC,DCA_CODE,DCA_DESC,LIGHT_CONDITION,...,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_NOT_INJ,NO_PERSONS,POLICE_ATTEND,ROAD_GEOMETRY,ROAD_GEOMETRY_DESC,SEVERITY,SPEED_ZONE,RMA
3,T20120000050,2012-01-01,15:00:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,1,...,0,1,0,1,1,5,Not at intersection,3,999,Local Road
13,T20120000602,2012-01-01,01:00:00,8,No collision and no object struck,1,Sunday,174,OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT),6,...,0,1,0,1,2,2,T intersection,3,999,Local Road
66,T20120000278,2012-01-03,17:25:00,1,Collision with vehicle,3,Tuesday,130,REAR END(VEHICLES IN SAME LANE),1,...,0,1,1,2,2,2,T intersection,3,999,Local Road
67,T20120004791,2012-01-03,16:15:00,1,Collision with vehicle,3,Tuesday,130,REAR END(VEHICLES IN SAME LANE),1,...,0,1,1,2,1,1,Cross intersection,3,999,Arterial Highway
68,T20120001375,2012-01-03,16:05:00,1,Collision with vehicle,3,Tuesday,111,RIGHT FAR (INTERSECTIONS ONLY),1,...,0,1,1,2,1,1,Cross intersection,3,999,Local Road
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167124,T20230025330,2023-09-30,12:30:00,8,No collision and no object struck,7,Saturday,184,OUT OF CONTROL ON CARRIAGEWAY (ON BEND),1,...,0,1,0,1,2,5,Not at intersection,3,999,Local Road
167131,T20230024738,2023-09-30,11:00:00,6,Vehicle overturned (no collision),7,Saturday,189,OTHER ACCIDENTS ON CURVE NOT INCLUDED IN DCAs ...,1,...,1,0,0,1,2,5,Not at intersection,2,888,Local Road
167136,T20230022975,2023-09-30,13:00:00,8,No collision and no object struck,7,Saturday,174,OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT),1,...,0,1,0,1,2,5,Not at intersection,3,888,Arterial Other
167142,T20230022861,2023-09-30,12:40:00,8,No collision and no object struck,6,Saturday,174,OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT),1,...,1,0,0,1,1,5,Not at intersection,2,888,Local Road


In [17]:
# Investigating RMA column
accident_df.loc[accident_df["RMA"].isnull()==True].head(3)

,ACCIDENT_NO,ACCIDENT_DATE,ACCIDENT_TIME,ACCIDENT_TYPE,ACCIDENT_TYPE_DESC,DAY_OF_WEEK,DAY_WEEK_DESC,DCA_CODE,DCA_DESC,LIGHT_CONDITION,...,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_NOT_INJ,NO_PERSONS,POLICE_ATTEND,ROAD_GEOMETRY,ROAD_GEOMETRY_DESC,SEVERITY,SPEED_ZONE,RMA
325,T20120001369,2012-01-12,21:00:00,6,Vehicle overturned (no collision),5,Thursday,174,OUT OF CONTROL ON CARRIAGEWAY (ON STRAIGHT),6,...,0,1,0,1,2,2,T intersection,3,999,NaN
329,T20120000924,2012-01-12,11:00:00,1,Collision with vehicle,5,Thursday,130,REAR END(VEHICLES IN SAME LANE),1,...,2,0,1,3,1,5,Not at intersection,2,100,NaN
942,T20120002126,2012-01-28,12:00:00,5,collision with some other object,7,Saturday,166,STRUCK OBJECT ON CARRIAGEWAY,1,...,1,0,0,1,1,5,Not at intersection,2,999,NaN


### Issues found in accidents_df 

##### Column DAY_OF_WEEK issue:

DAY_OF_WEEK does not align with which day it is for. 

Only day 6 and 7 correlate to one specifc day (saturday and sunday respectively)

<i>Data entry error? no information about what it should be.</i>

##### Column DAY_OF_WEEK fix:

<b>Remove DAY_OF_WEEK</b> as DAY_WEEK_DESC is all thats needed. 

##### Column DCA investigation results:
All DCA codes are correct according to the documented DCA codes and descriptions.

- cross reference with DCA code pdf


##### Column Speed zone Issue:

There are values of 777,888 & 999 which should not be possible, and too many rows of them to be considered an error. Prehaps there is a reasoning behind it. 

A total of 11932 rows affected 

777 has 301 rows 

888 has 1036 rows

999 has 10595 rows 

- prehaps another table will have an explanantion 

##### Column RMA issue:

RMA has NaN values but there no indications on what RMA could possible mean.

Unique values of 
"Local Road", "Arterial Highway", "Arterial Other", "Freeway","Nan"

Seems to refer to road classification (Road Mangement Act)?
https://www.vicroads.vic.gov.au/traffic-and-road-use/road-network-and-performance/victorias-road-network


### Fixes applied to accident_df

In [18]:
# Removing Day of Week 
accident_df = accident_df.drop(columns = ["DAY_OF_WEEK"])

accident_df.columns

Index(['ACCIDENT_NO', 'ACCIDENT_DATE', 'ACCIDENT_TIME', 'ACCIDENT_TYPE',
       'ACCIDENT_TYPE_DESC', 'DAY_WEEK_DESC', 'DCA_CODE', 'DCA_DESC',
       'LIGHT_CONDITION', 'NODE_ID', 'NO_OF_VEHICLES', 'NO_PERSONS_KILLED',
       'NO_PERSONS_INJ_2', 'NO_PERSONS_INJ_3', 'NO_PERSONS_NOT_INJ',
       'NO_PERSONS', 'POLICE_ATTEND', 'ROAD_GEOMETRY', 'ROAD_GEOMETRY_DESC',
       'SEVERITY', 'SPEED_ZONE', 'RMA'],
      dtype='object')

In [19]:
# Replace NaN values in RMA columns with N/A

accident_df["RMA"]=accident_df["RMA"].fillna("N/A")

accident_df["RMA"].isnull().any()

False

# Accident Location dataframe

In [20]:
# Accident Location first look
inital_shape_unique(accident_location_df)

Accident dataframe has a shape of (167153, 9)
ACCIDENT_NO            object
NODE_ID                 int64
ROAD_ROUTE_1          float64
ROAD_NAME              object
ROAD_TYPE              object
ROAD_NAME_INT          object
ROAD_TYPE_INT          object
DISTANCE_LOCATION     float64
DIRECTION_LOCATION     object
dtype: object
-----------------------------------
ACCIDENT_NO has 167153 unique values
['T20120000060' 'T20120000082' 'T20120000090' ... 'T20240003793'
 'T20240004249' 'T20240005783']
Null Presents: False
-----------------------------------
NODE_ID has 121039 unique values
[248602  41011 273187 ... 792196 792723 800569]
Null Presents: False
-----------------------------------
ROAD_ROUTE_1 has 1449 unique values
[9999. 5784. 2510. ... 7360. 7152. 7305.]
Null Presents: True
-----------------------------------
ROAD_NAME has 13306 unique values
['LINMAX' 'DORSET' 'CHURCHILL PARK' ... 'OLD LEONGATHA' 'MILL CREEK'
 'WEISS']
Null Presents: True
-----------------------------------
RO

# Accident Location Investigations

In [21]:
# Drop unused columns 

accident_location_df = accident_location_df.drop(["ROAD_ROUTE_1","NODE_ID","DISTANCE_LOCATION"],axis=1)

accident_location_df.head()

,ACCIDENT_NO,ROAD_NAME,ROAD_TYPE,ROAD_NAME_INT,ROAD_TYPE_INT,DIRECTION_LOCATION
0,T20120000060,LINMAX,COURT,WALLACE,AVENUE,NW
1,T20120000082,DORSET,ROAD,ALLAMBANAN,DRIVE,S
2,T20120000090,CHURCHILL PARK,DRIVE,PARKSIDE,BOULEVARD,E
3,T20120000093,PRINCES,HIGHWAY,IRVINES,ROAD,SW
4,T20120000104,DAVIS,ROAD,LEMARNE,ROAD,W


In [22]:
# Replace all null values and empty values with string NK (Not Known)
accident_location_df = accident_location_df.fillna("NK")
accident_location_df = accident_location_df.replace(to_replace=" ",value = "NK")

In [23]:
# Accident Location first look
inital_shape_unique(accident_location_df)

Accident dataframe has a shape of (167153, 6)
ACCIDENT_NO           object
ROAD_NAME             object
ROAD_TYPE             object
ROAD_NAME_INT         object
ROAD_TYPE_INT         object
DIRECTION_LOCATION    object
dtype: object
-----------------------------------
ACCIDENT_NO has 167153 unique values
['T20120000060' 'T20120000082' 'T20120000090' ... 'T20240003793'
 'T20240004249' 'T20240005783']
Null Presents: False
-----------------------------------
ROAD_NAME has 13306 unique values
['LINMAX' 'DORSET' 'CHURCHILL PARK' ... 'OLD LEONGATHA' 'MILL CREEK'
 'WEISS']
Null Presents: False
-----------------------------------
ROAD_TYPE has 85 unique values
['COURT' 'ROAD' 'DRIVE' 'HIGHWAY' 'STREET' 'TUNNEL' 'CRESCENT' 'AVENUE'
 'CLOSE' 'LANE' 'FREEWAY' 'TRACK' 'BOULEVARD' 'NK' 'PLACE' 'PARADE' 'WAY'
 'RAMP' 'GROVE' 'BYPASS' 'TERRACE' 'ESPLANADE' 'ALLEY' 'CAUSEWAY' 'GREEN'
 'QUADRANT' 'SQUARE' 'FIRELINE' 'CIRCUIT' 'WEST' 'WALK' 'RISE' 'LINK'
 'EAST' 'MEWS' 'BREAK' 'BOULEVARDE' 'BEND' 'PROM

# Accident Event dataframe

In [24]:
# Accident event first look
inital_shape_unique(accident_event_df)

Accident dataframe has a shape of (274938, 13)
ACCIDENT_NO               object
EVENT_SEQ_NO               int64
EVENT_TYPE                object
EVENT_TYPE_DESC           object
VEHICLE_1_ID              object
VEHICLE_1_COLL_PT         object
VEHICLE 1 COLL PT DESC    object
VEHICLE_2_ID              object
VEHICLE_2_COLL_PT         object
VEHICLE 2 COLL PT DESC    object
PERSON_ID                 object
OBJECT_TYPE                int64
OBJECT_TYPE_DESC          object
dtype: object
-----------------------------------
ACCIDENT_NO has 167149 unique values
['T20120003962' 'T20120003991' 'T20120003998' ... 'T20210025642'
 'T20210025658' 'T20210025691']
Null Presents: False
-----------------------------------
EVENT_SEQ_NO has 46 unique values
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Null Presents: False
-----------------------------------
EVENT_TYPE has 10 unique values
['C' '3' '2' '1' '8

In [25]:
# Changing null values 

accident_event_df[["VEHICLE 1 COLL PT DESC","VEHICLE 2 COLL PT DESC"]] = accident_event_df[["VEHICLE 1 COLL PT DESC","VEHICLE 2 COLL PT DESC"]].fillna("Not known or Not Applicable")

accident_event_df[["VEHICLE_2_ID","VEHICLE_2_COLL_PT","PERSON_ID"]] = accident_event_df[["VEHICLE_2_ID","VEHICLE_2_COLL_PT","PERSON_ID"]].fillna("NK")


# Atmoshoeric dataframe

In [26]:
inital_shape_unique(atmospheric_df)

Accident dataframe has a shape of (169367, 4)
ACCIDENT_NO          object
ATMOSPH_COND          int64
ATMOSPH_COND_SEQ      int64
ATMOSPH_COND_DESC    object
dtype: object
-----------------------------------
ACCIDENT_NO has 167153 unique values
['T20130006765' 'T20130006774' 'T20130006803' ... 'T20120013748'
 'T20120013790' 'T20120013827']
Null Presents: False
-----------------------------------
ATMOSPH_COND has 8 unique values
[1 9 4 2 7 5 6 3]
Null Presents: False
-----------------------------------
ATMOSPH_COND_SEQ has 5 unique values
[1 0 2 3 4]
Null Presents: False
-----------------------------------
ATMOSPH_COND_DESC has 8 unique values
['Clear' 'Not known' 'Fog' 'Raining' 'Strong winds' 'Smoke' 'Dust'
 'Snowing']
Null Presents: False


In [27]:
# Look up an accidents with multiple sequences
atmospheric_df.loc[atmospheric_df["ACCIDENT_NO"]=="T20190001830"]

,ACCIDENT_NO,ATMOSPH_COND,ATMOSPH_COND_SEQ,ATMOSPH_COND_DESC
91046,T20190001830,2,2,Raining
91047,T20190001830,5,3,Smoke
91048,T20190001830,6,1,Dust
91049,T20190001830,7,4,Strong winds


In [28]:
# Testing ATMOSPH_COND correlation with ATMOSPH_COND_DESC. 
# It matches up
atmospheric_df.loc[atmospheric_df["ATMOSPH_COND"]==9]["ATMOSPH_COND_DESC"].unique()

array(['Not known'], dtype=object)

### Issues in Atmoshpheric dataframe

Accidents that inolve a sequence have multiple entries for their different road conditions. possible issue in the future?

# Node dataframe

In [29]:
inital_shape_unique(node_df)

Accident dataframe has a shape of (175460, 10)
ACCIDENT_NO        object
NODE_ID             int64
NODE_TYPE          object
AMG_X             float64
AMG_Y             float64
LGA_NAME           object
DEG_URBAN_NAME     object
LATITUDE          float64
LONGITUDE         float64
POSTCODE_CRASH    float64
dtype: object
-----------------------------------
ACCIDENT_NO has 172678 unique values
['T20220021115' 'T20220016471' 'T20220011545' ... 'T20240008151'
 'T20230023107' 'T20230023059']
Null Presents: False
-----------------------------------
NODE_ID has 124958 unique values
[725433 726743 726774 ... 801031 801035 801036]
Null Presents: False
-----------------------------------
NODE_TYPE has 4 unique values
['N' 'O' 'I' ' ']
Null Presents: False
-----------------------------------
AMG_X has 124553 unique values
[2638081.59      2504398.399     2652318.1613286 ... 2504909.118
 2511021.713     2582792.656    ]
Null Presents: False
-----------------------------------
AMG_Y has 124545 uniqu

In [30]:
# Drop non-used columns 

node_df = node_df.drop(["NODE_ID","NODE_TYPE","AMG_X","AMG_Y"],axis=1)

In [31]:
node_df.loc[node_df["POSTCODE_CRASH"].isnull()]

,ACCIDENT_NO,LGA_NAME,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH
0,T20220021115,LATROBE,RURAL_VICTORIA,-38.188345,146.576115,NaN
1,T20220016471,STONNINGTON,MELB_URBAN,-37.866454,145.049989,NaN
2,T20220011545,WELLINGTON,RURAL_VICTORIA,-37.985367,146.733949,NaN
3,T20220009098,BENALLA,RURAL_VICTORIA,-36.797963,145.866604,NaN
4,T20220017520,GOLDEN PLAINS,SMALL_TOWNS,-38.021179,144.024292,NaN
...,...,...,...,...,...,...
175455,T20240006856,MOORABOOL,RURAL_VICTORIA,-37.669870,143.930038,NaN
175456,T20240008435,CARDINIA,RURAL_VICTORIA,-37.946829,145.653548,NaN
175457,T20240008151,BANYULE,MELB_URBAN,-37.756050,145.055711,NaN
175458,T20230023107,MONASH,MELB_URBAN,-37.892401,145.125307,NaN


In [32]:
target_df = node_df.loc[node_df["POSTCODE_CRASH"].isnull()]

target_df.iloc[0]["LONGITUDE"]

146.576115

In [33]:
#Define dataframe
target_df = node_df.loc[node_df["POSTCODE_CRASH"].isnull()]

In [34]:
# Set up geolocator 
geolocator = Nominatim(user_agent="postcode_data")

# Set up ratelimiter for geocoder
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)


In [ ]:
# Obtain postcode from geocoder database, May not run due to geocoder timeouts
for i in tqdm(range(0,len(target_df))):

    lat = target_df.iloc[i]["LATITUDE"]
    long = target_df.iloc[i]["LONGITUDE"]
    
    target_location = str(lat) +", "+ str(long)
    
    location = geolocator.reverse(target_location)
    
    if "postcode" in location.raw["address"]:
        postcode = location.raw["address"]["postcode"]
    else:
        postcode = "NK"
    
    postcode_list.append(postcode)


print(postcode_list)
print(len(postcode_list))
    

In [ ]:
# Save the postcode as a series to update node dataframe
target_df["POSTCODE_CRASH"] = postcode_list

In [ ]:
# Update node dataframe with the new postcodes
node_df.update(target_df["POSTCODE_CRASH"])

In [ ]:
# Checking unique values of postcode crash
print(list(node_df["POSTCODE_CRASH"].unique()))

In [ ]:
# Cleanning postcode crash string length

# Change column type to string
node_df["POSTCODE_CRASH"]= node_df["POSTCODE_CRASH"].astype(str)

# Cutting string down to 4 
node_df['POSTCODE_CRASH'] = node_df['POSTCODE_CRASH'].str.slice(0,4)

In [ ]:
# Rechecking postcode crash string length
print(list(node_df["POSTCODE_CRASH"].unique()))

### Issues in Node Dataframe

No context for Node columns

No use for AMG columns 

- Drop all useless columns later

Postcode missing for some rows 
- Manually entered postcode using geolocater 


# Person Dataframe


In [37]:
inital_shape_unique(person_df)

Accident dataframe has a shape of (390626, 14)
ACCIDENT_NO             object
PERSON_ID               object
VEHICLE_ID              object
SEX                     object
AGE_GROUP               object
INJ_LEVEL                int64
INJ_LEVEL_DESC          object
SEATING_POSITION        object
HELMET_BELT_WORN         int64
ROAD_USER_TYPE           int64
ROAD_USER_TYPE_DESC     object
LICENCE_STATE           object
TAKEN_HOSPITAL          object
EJECTED_CODE           float64
dtype: object
-----------------------------------
ACCIDENT_NO has 167153 unique values
['T20220027723' 'T20210018500' 'T20210026317' ... 'T20210023706'
 'T20230021583' 'T20230021516']
Null Presents: False
-----------------------------------
PERSON_ID has 116 unique values
['B' 'A' '01' '02' '03' 'C' '09' 'D' '04' '08' '05' '48' '19' 'F' '27'
 '12' '20' '17' '06' '11' 'E' '83' '10' '15' '14' '07' '34' '42' '13' 'H'
 'J' '16' '22' '26' '18' '41' '49' '31' '25' '29' '28' '35' '23' '21' '37'
 '38' '24' '40' 'Q' 'P' 'G

In [38]:
# Replacing null as Not Known 
person_df = person_df.fillna("NK")

### Issues in Person Dataframe

Nulls present
- Replace null with not known for all columns

Column ROAD_USER_TYPE has entries 2 & 7 that both describe "Drivers"
- Fix by replacing into type 2 

In [39]:
# Replacing 7 with 2 in road user type
person_df["ROAD_USER_TYPE"]=person_df["ROAD_USER_TYPE"].replace(to_replace=7,value=2)

In [40]:
# Checking replacments
person_df["ROAD_USER_TYPE"].unique()

array([2, 3, 4, 9, 1, 6, 8, 5])

# road surface dataframe

In [41]:
inital_shape_unique(road_surface_df)

Accident dataframe has a shape of (168097, 4)
ACCIDENT_NO          object
SURFACE_COND          int64
SURFACE_COND_DESC    object
SURFACE_COND_SEQ      int64
dtype: object
-----------------------------------
ACCIDENT_NO has 167153 unique values
['T20130012339' 'T20130012360' 'T20130012370' ... 'T20130012310'
 'T20130012312' 'T20130012328']
Null Presents: False
-----------------------------------
SURFACE_COND has 6 unique values
[9 1 2 5 3 4]
Null Presents: False
-----------------------------------
SURFACE_COND_DESC has 6 unique values
['Unk.' 'Dry' 'Wet' 'Icy' 'Muddy' 'Snowy']
Null Presents: False
-----------------------------------
SURFACE_COND_SEQ has 4 unique values
[1 0 2 3]
Null Presents: False


# sub_dca_df.head()

In [42]:
inital_shape_unique(sub_dca_df)

Accident dataframe has a shape of (239313, 4)
ACCIDENT_NO          object
SUB_DCA_CODE         object
SUB_DCA_SEQ           int64
SUB_DCA_CODE_DESC    object
dtype: object
-----------------------------------
ACCIDENT_NO has 166279 unique values
['T20120023069' 'T20120023107' 'T20120023147' ... 'T20220024633'
 'T20220024960' 'T20220025613']
Null Presents: False
-----------------------------------
SUB_DCA_CODE has 112 unique values
['NRQ' 'L04' 'N01' 'Q04' 'Q06' 'A02' 'L03' 'I05' 'V03' 'R03' 'Q36' 'Q34'
 'G02' 'A01' 'R02' 'N02' 'Z06' 'M01' 'G03' 'Q24' 'Z08' 'K01' 'Q13' 'Q23'
 'B04' 'V01' 'Q01' 'L01' 'Z09' 'S03' 'F03' 'F04' 'J02' 'E05' 'C02' 'Q08'
 'Q11' 'V06' 'Q37' 'B01' 'Q09' 'Q07' 'V05' 'D01' 'Q32' 'Q12' 'I01' 'B03'
 'H03' 'Z07' 'A04' 'V07' 'R05' 'Z05' 'Q02' 'V02' 'Y01' 'Q21' 'F02' 'Q03'
 'Z01' 'I04' 'S02' 'S01' 'X02' 'A03' 'P02' 'I06' 'G01' 'C01' 'E03' 'Q17'
 'Q38' 'Q33' 'I02' 'R04' 'D07' 'H01' 'W02' 'P01' 'H04' 'J01' 'B02' 'E04'
 'X01' 'Z04' 'Z02' 'D04' 'Q31' 'D02' 'E06' 'K02' 'Q05' 

# vehicle_df

In [43]:
inital_shape_unique(vehicle_df)

Accident dataframe has a shape of (304729, 37)
ACCIDENT_NO                object
VEHICLE_ID                 object
VEHICLE_YEAR_MANUF        float64
VEHICLE_DCA_CODE          float64
INITIAL_DIRECTION          object
ROAD_SURFACE_TYPE         float64
ROAD_SURFACE_TYPE_DESC     object
REG_STATE                  object
VEHICLE_BODY_STYLE         object
VEHICLE_MAKE               object
VEHICLE_MODEL              object
VEHICLE_POWER             float64
VEHICLE_TYPE                int64
VEHICLE_TYPE_DESC          object
VEHICLE_WEIGHT            float64
CONSTRUCTION_TYPE          object
FUEL_TYPE                  object
NO_OF_WHEELS              float64
NO_OF_CYLINDERS           float64
SEATING_CAPACITY          float64
TARE_WEIGHT               float64
TOTAL_NO_OCCUPANTS        float64
CARRY_CAPACITY            float64
CUBIC_CAPACITY            float64
FINAL_DIRECTION            object
DRIVER_INTENT             float64
VEHICLE_MOVEMENT          float64
TRAILER_TYPE               object
V

VEHICLE_POWER has 0 unique values
[nan]
Null Presents: True
-----------------------------------
VEHICLE_TYPE has 28 unique values
[ 1  2  3 13 10  4  5 60 72 99 71 62 17 11 12 61 15  8  9 16 27 18 20 19
 63 14  6  7]
Null Presents: False
-----------------------------------
VEHICLE_TYPE_DESC has 28 unique values
['Car' 'Station Wagon' 'Taxi' 'Bicycle' 'Motor Cycle' 'Utility'
 'Panel Van' 'Prime Mover Only' 'Heavy Vehicle (Rigid) > 4.5 Tonnes'
 'Not Known' 'Light Commercial Vehicle (Rigid) <= 4.5 Tonnes GVM'
 'Prime Mover B-Double' 'Other Vehicle' 'Moped' 'Motor Scooter'
 'Prime Mover - Single Trailer' 'Tram' 'Bus/Coach' 'Mini Bus(9-13 seats)'
 'Train' 'Plant machinery and Agricultural equipment' 'Not Applicable'
 'Quad Bike' 'Parked trailers' 'Prime Mover B-Triple'
 'Horse (ridden or drawn)' 'Prime Mover (No of Trailers Unknown)'
 'Rigid Truck(Weight Unknown)']
Null Presents: False
-----------------------------------
VEHICLE_WEIGHT has 1224 unique values
[   nan  1450.  3260. ... 42000.

['WHI' 'BLU' 'GLD' 'BLK' 'YLW' 'SIL' 'ZZ' 'GRY' 'GRN' 'OGE' 'PUR' 'BRN'
 'RED' 'MRN' 'FWN' 'CRM' 'MVE' 'PNK' nan]
Null Presents: True
-----------------------------------
VEHICLE_COLOUR_2 has 17 unique values
['ZZ' 'WHI' 'SIL' 'MRN' 'GRN' 'RED' 'BLU' 'OGE' 'BLK' 'GRY' 'YLW' 'GLD'
 'BRN' 'CRM' 'FWN' 'PUR' 'PNK' nan]
Null Presents: True
-----------------------------------
CAUGHT_FIRE has 4 unique values
[ 2.  0.  1.  9. nan]
Null Presents: True
-----------------------------------
INITIAL_IMPACT has 16 unique values
['1' '5' '4' 'F' '6' 'N' '8' '9' 'T' '2' 'U' '7' '3' 'R' '0' 'S' nan]
Null Presents: True
-----------------------------------
LAMPS has 4 unique values
[ 1.  9.  2.  0. nan]
Null Presents: True
-----------------------------------
LEVEL_OF_DAMAGE has 7 unique values
[5 2 4 3 6 1 9]
Null Presents: False
-----------------------------------
TOWED_AWAY_FLAG has 3 unique values
[ 1.  2.  9. nan]
Null Presents: True
-----------------------------------
TRAFFIC_CONTROL has 17 unique val

# Vehicle investigation

In [44]:
# Drop columns
vehicle_df = vehicle_df.drop(["VEHICLE_POWER","CONSTRUCTION_TYPE","TARE_WEIGHT","CUBIC_CAPACITY"],axis=1)

In [45]:
vehicle_df["ROAD_SURFACE_TYPE_DESC"] = vehicle_df["ROAD_SURFACE_TYPE_DESC"].fillna("Not Known")
vehicle_df["TRAFFIC_CONTROL_DESC"] = vehicle_df["TRAFFIC_CONTROL_DESC"].fillna("Unknown")
vehicle_df[["VEHICLE_YEAR_MANUF","VEHICLE_DCA_CODE","ROAD_SURFACE_TYPE","VEHICLE_WEIGHT",
            "NO_OF_CYLINDERS","SEATING_CAPACITY","TOTAL_NO_OCCUPANTS","CARRY_CAPACITY",
            "DRIVER_INTENT","VEHICLE_MOVEMENT","CAUGHT_FIRE","LAMPS","TOWED_AWAY_FLAG","NO_OF_WHEELS","TRAFFIC_CONTROL"]] = vehicle_df[["VEHICLE_YEAR_MANUF","VEHICLE_DCA_CODE","ROAD_SURFACE_TYPE","VEHICLE_WEIGHT",
            "NO_OF_CYLINDERS","SEATING_CAPACITY","TOTAL_NO_OCCUPANTS","CARRY_CAPACITY",
            "DRIVER_INTENT","VEHICLE_MOVEMENT","CAUGHT_FIRE","LAMPS","TOWED_AWAY_FLAG","NO_OF_WHEELS",
            "TRAFFIC_CONTROL"]].fillna(0)

vehicle_df[["INITIAL_DIRECTION","REG_STATE","VEHICLE_BODY_STYLE","VEHICLE_MAKE","VEHICLE_MODEL","FUEL_TYPE",
           "FINAL_DIRECTION","VEHICLE_COLOUR_1","VEHICLE_COLOUR_2","TRAILER_TYPE"]] =vehicle_df[["INITIAL_DIRECTION","REG_STATE","VEHICLE_BODY_STYLE","VEHICLE_MAKE","VEHICLE_MODEL","FUEL_TYPE",
           "FINAL_DIRECTION","VEHICLE_COLOUR_1","VEHICLE_COLOUR_2","TRAILER_TYPE"]].fillna("NK")

vehicle_df["INITIAL_IMPACT"] = vehicle_df["INITIAL_IMPACT"].fillna("0")

In [46]:
inital_shape_unique(vehicle_df)

Accident dataframe has a shape of (304729, 33)
ACCIDENT_NO                object
VEHICLE_ID                 object
VEHICLE_YEAR_MANUF        float64
VEHICLE_DCA_CODE          float64
INITIAL_DIRECTION          object
ROAD_SURFACE_TYPE         float64
ROAD_SURFACE_TYPE_DESC     object
REG_STATE                  object
VEHICLE_BODY_STYLE         object
VEHICLE_MAKE               object
VEHICLE_MODEL              object
VEHICLE_TYPE                int64
VEHICLE_TYPE_DESC          object
VEHICLE_WEIGHT            float64
FUEL_TYPE                  object
NO_OF_WHEELS              float64
NO_OF_CYLINDERS           float64
SEATING_CAPACITY          float64
TOTAL_NO_OCCUPANTS        float64
CARRY_CAPACITY            float64
FINAL_DIRECTION            object
DRIVER_INTENT             float64
VEHICLE_MOVEMENT          float64
TRAILER_TYPE               object
VEHICLE_COLOUR_1           object
VEHICLE_COLOUR_2           object
CAUGHT_FIRE               float64
INITIAL_IMPACT             object
L

VEHICLE_COLOUR_1 has 19 unique values
['WHI' 'BLU' 'GLD' 'BLK' 'YLW' 'SIL' 'ZZ' 'GRY' 'GRN' 'OGE' 'PUR' 'BRN'
 'RED' 'MRN' 'FWN' 'CRM' 'MVE' 'PNK' 'NK']
Null Presents: False
-----------------------------------
VEHICLE_COLOUR_2 has 18 unique values
['ZZ' 'WHI' 'SIL' 'MRN' 'GRN' 'RED' 'BLU' 'OGE' 'BLK' 'GRY' 'YLW' 'GLD'
 'BRN' 'CRM' 'FWN' 'PUR' 'PNK' 'NK']
Null Presents: False
-----------------------------------
CAUGHT_FIRE has 4 unique values
[2. 0. 1. 9.]
Null Presents: False
-----------------------------------
INITIAL_IMPACT has 16 unique values
['1' '5' '4' 'F' '6' 'N' '8' '9' 'T' '2' 'U' '7' '3' 'R' '0' 'S']
Null Presents: False
-----------------------------------
LAMPS has 4 unique values
[1. 9. 2. 0.]
Null Presents: False
-----------------------------------
LEVEL_OF_DAMAGE has 7 unique values
[5 2 4 3 6 1 9]
Null Presents: False
-----------------------------------
TOWED_AWAY_FLAG has 4 unique values
[1. 2. 9. 0.]
Null Presents: False
-----------------------------------
TRAFFIC_CON

In [47]:
inital_shape_unique(vehicle_df )
accident_df
accident_event_df
accident_location_df 
atmospheric_df
node_df
person_df
road_surface_df 
sub_dca_df 
vehicle_df 

Accident dataframe has a shape of (304729, 33)
ACCIDENT_NO                object
VEHICLE_ID                 object
VEHICLE_YEAR_MANUF        float64
VEHICLE_DCA_CODE          float64
INITIAL_DIRECTION          object
ROAD_SURFACE_TYPE         float64
ROAD_SURFACE_TYPE_DESC     object
REG_STATE                  object
VEHICLE_BODY_STYLE         object
VEHICLE_MAKE               object
VEHICLE_MODEL              object
VEHICLE_TYPE                int64
VEHICLE_TYPE_DESC          object
VEHICLE_WEIGHT            float64
FUEL_TYPE                  object
NO_OF_WHEELS              float64
NO_OF_CYLINDERS           float64
SEATING_CAPACITY          float64
TOTAL_NO_OCCUPANTS        float64
CARRY_CAPACITY            float64
FINAL_DIRECTION            object
DRIVER_INTENT             float64
VEHICLE_MOVEMENT          float64
TRAILER_TYPE               object
VEHICLE_COLOUR_1           object
VEHICLE_COLOUR_2           object
CAUGHT_FIRE               float64
INITIAL_IMPACT             object
L

VEHICLE_TYPE_DESC has 28 unique values
['Car' 'Station Wagon' 'Taxi' 'Bicycle' 'Motor Cycle' 'Utility'
 'Panel Van' 'Prime Mover Only' 'Heavy Vehicle (Rigid) > 4.5 Tonnes'
 'Not Known' 'Light Commercial Vehicle (Rigid) <= 4.5 Tonnes GVM'
 'Prime Mover B-Double' 'Other Vehicle' 'Moped' 'Motor Scooter'
 'Prime Mover - Single Trailer' 'Tram' 'Bus/Coach' 'Mini Bus(9-13 seats)'
 'Train' 'Plant machinery and Agricultural equipment' 'Not Applicable'
 'Quad Bike' 'Parked trailers' 'Prime Mover B-Triple'
 'Horse (ridden or drawn)' 'Prime Mover (No of Trailers Unknown)'
 'Rigid Truck(Weight Unknown)']
Null Presents: False
-----------------------------------
VEHICLE_WEIGHT has 1225 unique values
[    0.  1450.  3260. ... 42000.  1430.  2264.]
Null Presents: False
-----------------------------------
FUEL_TYPE has 9 unique values
['P' 'Z' 'M' 'D' 'NK' 'G' 'E' 'S' 'O']
Null Presents: False
-----------------------------------
NO_OF_WHEELS has 15 unique values
[ 4.  0.  2.  6. 10.  8.  5.  3.  7. 46. 

,ACCIDENT_NO,VEHICLE_ID,VEHICLE_YEAR_MANUF,VEHICLE_DCA_CODE,INITIAL_DIRECTION,ROAD_SURFACE_TYPE,ROAD_SURFACE_TYPE_DESC,REG_STATE,VEHICLE_BODY_STYLE,VEHICLE_MAKE,...,TRAILER_TYPE,VEHICLE_COLOUR_1,VEHICLE_COLOUR_2,CAUGHT_FIRE,INITIAL_IMPACT,LAMPS,LEVEL_OF_DAMAGE,TOWED_AWAY_FLAG,TRAFFIC_CONTROL,TRAFFIC_CONTROL_DESC
0,T20120000009,A,1996.0,1.0,E,1.0,Paved,V,SEDAN,HOLDEN,...,H,WHI,ZZ,2.0,1,1.0,5,1.0,0.0,No control
1,T20120000012,A,2002.0,1.0,W,1.0,Paved,V,SEDAN,HOLDEN,...,H,BLU,ZZ,2.0,1,1.0,2,2.0,1.0,Stop-go lights
2,T20120000012,B,1988.0,2.0,S,1.0,Paved,V,SEDAN,TOYOTA,...,H,WHI,ZZ,2.0,5,9.0,4,1.0,1.0,Stop-go lights
3,T20120000013,A,1997.0,1.0,N,1.0,Paved,V,SEDAN,MITSUB,...,H,GLD,ZZ,2.0,5,1.0,4,1.0,0.0,No control
4,T20120000013,B,2010.0,2.0,N,1.0,Paved,V,WAGON,TOYOTA,...,H,BLK,ZZ,2.0,4,2.0,4,2.0,0.0,No control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304724,T20240006909,B,2018.0,1.0,W,1.0,Paved,V,SEDAN,SUZUKI,...,H,WHI,ZZ,2.0,5,2.0,1,2.0,0.0,No control
304725,T20240006923,A,2012.0,1.0,S,9.0,Not known,V,WAGON,JEEP,...,H,BLK,ZZ,2.0,F,2.0,5,2.0,99.0,Unknown
304726,T20240006923,B,2010.0,2.0,N,9.0,Not known,V,SEDAN,FORD,...,H,SIL,ZZ,2.0,F,2.0,5,2.0,99.0,Unknown
304727,T20240006955,A,2005.0,2.0,W,1.0,Paved,V,SEDAN,TOYOTA,...,H,BLU,ZZ,2.0,F,2.0,6,2.0,5.0,Ped. crossing


In [48]:
# Saving dataframe into csv

outdir = './Resources/Cleanned'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
accident_df.to_csv('./Resources/Cleanned/accident_df.csv', index= False )
accident_event_df.to_csv('./Resources/Cleanned/accident_event_df.csv', index= False)
accident_location_df.to_csv('./Resources/Cleanned/accident_location_df.csv', index= False)
atmospheric_df.to_csv('./Resources/Cleanned/atmospheric_df.csv', index= False)
node_df.to_csv('./Resources/Cleanned/node_df.csv', index= False)
person_df.to_csv('./Resources/Cleanned/person_df.csv', index= False)
road_surface_df.to_csv('./Resources/Cleanned/road_surface_df.csv', index= False)
sub_dca_df.to_csv('./Resources/Cleanned/sub_dca_df.csv', index= False)
vehicle_df.to_csv('./Resources/Cleanned/vehicle_df.csv', index= False)
